In [4]:
import rasterio
from pyproj import Transformer
from pyproj import CRS
target_crs = CRS.from_proj4("+proj=sinu +R=6371007.181 +nadgrids=@null +units=m +no_defs")

def calculate_pixel_size_in_custom_proj(raster_path, target_crs_wkt):
    # 打开栅格文件
    with rasterio.open(raster_path) as src:
        # 获取仿射变换参数（其中 transform.a 表示 x 分辨率，-transform.e 表示 y 分辨率）
        transform = src.transform
        pixel_size_x = transform.a
        pixel_size_y = -transform.e

        # 获取栅格的中心像元坐标（行、列）
        height, width = src.height, src.width
        center_row = height // 2
        center_col = width // 2

        # 获取中心像元的地理坐标（注意返回顺序通常为 (x, y)）
        x_center, y_center = src.xy(center_row, center_col)
        src_crs = src.crs

    # 创建 Transformer：从原始 CRS 到目标自定义 CRS
    transformer = Transformer.from_crs(src_crs, target_crs, always_xy=True)

    # 转换中心点
    x0, y0 = transformer.transform(x_center, y_center)
    # 转换中心点右侧的点（增加一个像元大小）
    x1, y1 = transformer.transform(x_center + pixel_size_x, y_center)
    # 转换中心点上方的点（增加一个像元大小；注意 y 方向）
    x2, y2 = transformer.transform(x_center, y_center + pixel_size_y)

    # 计算转换后 x 方向和 y 方向的距离差（单位：米）
    pixel_size_meters_x = abs(x1 - x0)
    pixel_size_meters_y = abs(y2 - y0)

    return pixel_size_meters_x, pixel_size_meters_y

# 示例使用
raster_path2 = r'F:\BUILT_S\GHS_BUILT_S_E2018_GLOBE_R2023A_54009_10_V1_0.tif'
target_crs_wkt = (
    "PROJCS['unnamed_ellipse_Sinusoidal',GEOGCS['GCS_unnamed_ellipse',"
    "DATUM['D_unknown',SPHEROID['Unknown',6371007.181,0.0]],"
    "PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],"
    "PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],"
    "PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],"
    "UNIT['Meter',1.0]]"
)

px_x, px_y = calculate_pixel_size_in_custom_proj(raster_path2, target_crs_wkt)
print("Pixel size in custom projection: X = {} m, Y = {} m".format(px_x, px_y))


Pixel size in custom projection: X = 11.094791121351982 m, Y = 8.993098949440206 m


In [5]:
import rasterio
from pyproj import Transformer, CRS

# 定义目标坐标系（使用 PROJ 字符串）
target_crs = CRS.from_proj4("+proj=sinu +R=6371007.181 +nadgrids=@null +units=m +no_defs")

def calculate_pixel_area_in_custom_proj(raster_path):
    """
    计算栅格在目标自定义 Sinusoidal 投影下（假设像元为平行四边形）的像元宽度、高度及面积
    """
    with rasterio.open(raster_path) as src:
        # 获取仿射变换参数
        transform = src.transform
        pixel_size_x = transform.a        # x 分辨率
        pixel_size_y = -transform.e       # y 分辨率（取正）

        # 获取栅格的行列数以及中心像元位置
        height, width = src.height, src.width
        center_row = height // 2
        center_col = width // 2

        # 获取中心像元在源 CRS 中的坐标（通常为 (x, y)）
        x_center, y_center = src.xy(center_row, center_col)
        src_crs = src.crs

    # 创建 Transformer，将源 CRS 转换到目标自定义 Sinusoidal 投影
    transformer = Transformer.from_crs(src_crs, target_crs, always_xy=True)

    # 转换中心像元坐标
    x0, y0 = transformer.transform(x_center, y_center)
    # 转换中心像元右侧的点（增加一个像元宽度）
    x1, y1 = transformer.transform(x_center + pixel_size_x, y_center)
    # 转换中心像元上方的点（增加一个像元高度）
    x2, y2 = transformer.transform(x_center, y_center + pixel_size_y)

    # 计算转换后在 x 和 y 方向的距离差（近似像元宽度和高度）
    pixel_width = abs(x1 - x0)
    pixel_height = abs(y2 - y0)

    # 计算像元面积（平行四边形假设）
    pixel_area = pixel_width * pixel_height

    return pixel_width, pixel_height, pixel_area

# 示例使用
raster_path2 = r'F:\BUILT_S\GHS_BUILT_S_E2018_GLOBE_R2023A_54009_10_V1_0.tif'
width_m, height_m, area_m2 = calculate_pixel_area_in_custom_proj(raster_path2)
print("Pixel width: {} m, Pixel height: {} m, Pixel area: {} m^2".format(width_m, height_m, area_m2))


Pixel width: 11.094791121351982 m, Pixel height: 8.993098949440206 m, Pixel area: 99.77655437768904 m^2


In [6]:
import rasterio

file_path = r"F:\BUILT_S\GHS_BUILT_S_E2018_GLOBE_R2023A_54009_10_V1_0.tif"

with rasterio.open(file_path) as src:
    data = src.read(1)  # 读取第1个波段
    print("最小值:", data.min())
    print("最大值:", data.max())
    print("平均值:", data.mean())


MemoryError: Unable to allocate 5.91 TiB for an array with shape (1, 1800000, 3608200) and data type uint8